In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\MLflow-deployment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Desktop\\MLflow-deployment'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
# entitiy
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [7]:
# configuration

class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([Path(self.config["artifacts_root"])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]
        create_directories([Path(config["root_dir"])])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config["root_dir"]),
            source_URL=config["source_URL"],
            local_data_file=Path(config["local_data_file"]),
            unzip_dir=Path(config["unzip_dir"]),
        )
        return data_ingestion_config

In [8]:
#components

import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_data(self) -> Path:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        return Path(self.config.local_data_file)
    
    def extract_zip_file(self, zip_file_path: Path) -> Path:
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  # always ensure dir exists

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted to {unzip_path}")

        return unzip_path


In [10]:

#pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    zip_file_path = data_ingestion.download_data()
    data_ingestion.extract_zip_file(zip_file_path=zip_file_path)
except Exception as e:
    raise e

2025-10-04 17:34:15,738 - INFO - yaml file: config\config.yaml loaded successfully
2025-10-04 17:34:15,742 - INFO - yaml file: params.yaml loaded successfully
2025-10-04 17:34:15,747 - INFO - yaml file: schema.yaml loaded successfully
2025-10-04 17:34:15,747 - INFO - Directory created at: artifacts
2025-10-04 17:34:15,751 - INFO - Directory created at: artifacts\data_ingestion
2025-10-04 17:34:15,751 - INFO - File already exists of size: 22.78 KB
2025-10-04 17:34:15,819 - INFO - File extracted to artifacts\data_ingestion
